In [15]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week9/소설 작가 분류 AI 경진대회/train.csv') 

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


#1. keras Embedding Layer

In [17]:
max_len = 100
vocabulary = vectorizer.word_index

In [18]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = max_len))

#2. 

In [19]:
import os
os.chdir('/content/drive/MyDrive/ESAA(22-1)/Week10')

In [20]:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)

In [31]:
import numpy as np
embedding_matrix = np.zeros((vocab_size,300)) #300차원의 임베딩 매트릭스 생성

for index,word in enumerate(vocabulary):  #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vect 에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word]  #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector  #해당 위치의 embedding_matrix에 저장한다.
    else:
        print('word2vect에 없는 단어입니다.')
        break

word2vect에 없는 단어입니다.


#3. glove

In [23]:
import os
os.chdir('/content/drive/MyDrive/ESAA(22-1)/Week10/word-embeddings/word-embeddings')

In [12]:
2. # load the whole embedding into memory
import numpy as np
glove = dict()
f = open('/content/drive/MyDrive/ESAA(22-1)/Week10/word-embeddings/word-embeddings/glove/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector
f.close()

In [27]:
vocabulary

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'a': 5,
 'i': 6,
 'in': 7,
 'odin': 8,
 '”': 9,
 'he': 10,
 'was': 11,
 'it': 12,
 'you': 13,
 'that': 14,
 'his': 15,
 'with': 16,
 'had': 17,
 'for': 18,
 'as': 19,
 'her': 20,
 'at': 21,
 'not': 22,
 'my': 23,
 'is': 24,
 'but': 25,
 'have': 26,
 'be': 27,
 'she': 28,
 'me': 29,
 'him': 30,
 'said': 31,
 'on': 32,
 'all': 33,
 'so': 34,
 'this': 35,
 'from': 36,
 'by': 37,
 'which': 38,
 'were': 39,
 '’': 40,
 'there': 41,
 'no': 42,
 'one': 43,
 'what': 44,
 'been': 45,
 'would': 46,
 'we': 47,
 'they': 48,
 'are': 49,
 'your': 50,
 'if': 51,
 'an': 52,
 'very': 53,
 'mr': 54,
 'could': 55,
 'do': 56,
 'out': 57,
 'will': 58,
 'when': 59,
 'up': 60,
 'or': 61,
 'upon': 62,
 'them': 63,
 'more': 64,
 'man': 65,
 'now': 66,
 'who': 67,
 'some': 68,
 "'": 69,
 'am': 70,
 'know': 71,
 '“i': 72,
 'then': 73,
 'into': 74,
 'about': 75,
 'time': 76,
 'little': 77,
 'their': 78,
 'did': 79,
 'only': 80,
 'see': 81,
 'like': 82,
 'before': 83,
 'sh

In [28]:
# dict 형태의 자료에 enumerate를 적용할 경우
a = {'hello':1, 'world':2, 'Esaa':5, 'statistics':10}
for index,word in enumerate(a):
  print(index,'/',word)

0 / hello
1 / world
2 / Esaa
3 / statistics


In [34]:
embedding_matrix = np.zeros((vocab_size, 100)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vocabulary): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("'"+word+"'는 glove에 없는 단어입니다.")
        break

'odin'는 glove에 없는 단어입니다.


In [36]:
model = Sequential()
model.add(Embedding(vocab_size, 100,weights = [embedding_matrix], input_length = max_len))

#4. Fasttext

In [37]:
from gensim.models.keyedvectors import KeyedVectors
FastText = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/ESAA(22-1)/Week10/word-embeddings/word-embeddings/fasttext/fasttext.vec', binary = True,unicode_errors='ignore')

In [40]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vocabulary): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.

In [42]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = max_len))

#5. 대회 적용

##간단한 전처리 + 형태소 분석

In [46]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 3.8 MB/s 
     |████████████████████████████████| 448 kB 51.0 MB/s 


In [47]:
from konlpy.tag import Okt
import re
import tqdm 

In [51]:
from konlpy.tag import Okt
import re
import tqdm 

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Okt() #형태소 분석기 
    token_list = []
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) 
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문 
train['token'], okt = text_preprocessing(train['text'])

100%|██████████| 54879/54879 [06:34<00:00, 138.98it/s]


##Vectorization

In [52]:
def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1 #모델에게 알려줄 vocabulary의 크기 계산 
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    return X_train, vocab_size, tokenizer

train_y = train['author']
train_X, vocab_size, vectorizer = text2sequence(train['token'], max_len = 100)
print(train_X.shape, train_y.shape)

vocab_size :  36342
(54879, 100) (54879,)


##Embedding

In [54]:
import os
os.chdir('/content/drive/MyDrive/ESAA(22-1)/Week10')

In [ ]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300)) 

In [56]:
for index, word in enumerate(vocabulary): 
    if word in word2vec:  
        embedding_vector = word2vec[word] 
        embedding_matrix[index] = embedding_vector
    else:
        print("word2vec에 없는 단어입니다.")
        break

word2vec에 없는 단어입니다.


##Modeling

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding

def LSTM(vocab_size, max_len=1000):
    model = Sequential()
    #Sequential 모델은 층을 순서대로 쌓은 것으로, 아래와 같이 각 층 인스턴스를 리스트 형식으로 나열

    model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) #임베딩 가중치 적용 코드
    model.add(SpatialDropout1D(0.3)) #입력 벡터 중 30%만큼 층의 출력을 줄여
    model.add(LSTM(64)) #64 차원의 단일 벡터를 반환
    model.add(Dropout(0.5)) #학습과정 중 각 업데이트에서 임의로 인풋유닛을 0으로 설정하는 비율 0.5로 설정
    model.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001))) #64차원 출력에 'relu' 함수를 적용
    model.add(Dense(1, activation='sigmoid')) #1차원 출력에 'sigmoid' 함수를 적용
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model